In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [51]:
data = pd.read_csv("data/clean_2017-2019.csv");

In [10]:
d2 = pd.read_csv("data/imputed_data_r1.csv");

In [4]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'age', 'country_live',
       'country_work', 'emp_mh_resources', 'ever_diagnosed', 'family_history',
       'gender', 'identified', 'identifited_neg_consequences', 'interview_why',
       'know_resources', 'medical_coverage', 'mention_interview',
       'mh_benefits', 'mh_with_coworkers', 'mh_with_supervisor',
       'num_employees', 'past_disorder', 'ph_or_mh_discuss', 'primarily_tech',
       'primary_role_tech', 'race', 'reveal_to_clients', 'self-employed',
       'share_with_f&f', 'sought_treatment', 'state_live', 'state_work',
       'year'],
      dtype='object')

In [88]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,age,country_live,country_work,emp_mh_resources,ever_diagnosed,family_history,gender,...,primarily_tech,primary_role_tech,race,reveal_to_clients,self-employed,share_with_f&f,sought_treatment,state_live,state_work,year
0,0,0,0,27.0,United Kingdom,United Kingdom,I don't know,NaN,No,F,...,False,True,NaN,NaN,False,5,False,NaN,NaN,2017.0
1,1,1,1,31.0,United Kingdom,United Kingdom,No,NaN,No,M,...,False,True,NaN,NaN,False,4,False,NaN,NaN,2017.0
2,2,2,2,36.0,United States of America,United States of America,No,True,Yes,M,...,False,True,White,NaN,False,5,False,Missouri,Missouri,2017.0
3,3,3,3,22.0,United States of America,United States of America,I don't know,True,I don't know,M,...,False,True,White,NaN,False,10,False,Washington,Washington,2017.0
4,4,4,4,52.0,United States of America,United States of America,NaN,NaN,Yes,F,...,NaN,NaN,More than one,"Sometimes, if it comes up",False,10,False,Illinois,Illinois,2017.0


In [92]:
data['identified']

0       0.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
1520    0.0
1521    0.0
1522    0.0
1523    0.0
1524    1.0
Name: identified, Length: 1525, dtype: float64

In [5]:
# Show the number of NaN values for all columns:
for label in data.columns[3:]:
    print(label, data[label].isnull().sum())

age 2
country_live 2
country_work 2
emp_mh_resources 217
ever_diagnosed 863
family_history 0
gender 31
identified 2
identifited_neg_consequences 1345
interview_why 327
know_resources 1308
medical_coverage 1308
mention_interview 0
mh_benefits 217
mh_with_coworkers 217
mh_with_supervisor 217
num_employees 217
past_disorder 761
ph_or_mh_discuss 217
primarily_tech 217
primary_role_tech 217
race 566
reveal_to_clients 1308
self-employed 0
share_with_f&f 0
sought_treatment 0
state_live 513
state_work 504
year 0


In [114]:
labels_left_alone = ['country_live','country_work','state_live','state_work','race','gender','year', 'interview_why']
labels_real = ['age']
labels_bool = ['ever_diagnosed', 'identified', 'identifited_neg_consequences', 'medical_coverage', 'primarily_tech', 
    'primary_role_tech', 'self-employed', 'sought_treatment']

labels_cat = ['family_history', 'mention_interview', 'know_resources', 'emp_mh_resources', 'mh_benefits' ,'mh_with_coworkers', 
    'mh_with_supervisor', 'past_disorder', 'ph_or_mh_discuss', 'reveal_to_clients']
labels_ord = ['share_with_f&f', 'num_employees']

In [115]:
def str_to_bool(st):
    if pd.isna(st):
        return st
    else:
        if st in ['Yes', 'True','true','1.0']:
            return True
        else:
            return False

In [116]:
for i in range(len(labels_bool)):
    data[labels_bool[i]] = data[labels_bool[i]].apply(str_to_bool) 

In [117]:
# Ordinal encoder for the ordinal data:
ord_df = data[labels_ord].replace(np.nan, 'None') # change all NaN to None and then can manually change to NaN values
ord_enc = preprocessing.OrdinalEncoder(categories = [[0,1,2,3,4,5,6,7,8,9,10],['1-5','6-25','26-100','100-500','500-1000','More than 1000','None']] )
ord_df_enc = ord_enc.fit_transform(ord_df)
ord_df_enc[ord_df_enc[:,1]==6,1] = np.nan # change all None items to NaN for imputation
ord_df_enc = pd.DataFrame(ord_df_enc, columns = labels_ord)

In [118]:
ord_df_enc.head()

,share_with_f&f,num_employees
0,5.0,3.0
1,4.0,3.0
2,5.0,1.0
3,10.0,5.0
4,10.0,NaN


In [119]:
bool_df = data[labels_bool]
real_df = data[labels_real]
# Concatenate in boolean and real data to do imputation all together
knn_data = pd.concat([ord_df_enc, bool_df, real_df], axis=1)

knn_imputed = KNNImputer(n_neighbors=3).fit_transform(knn_data)
knn_imp_df = pd.DataFrame(knn_imputed.round(), columns = labels_ord+labels_bool+labels_real)

# Convert back to ordinal categories:
knn_imp_df[labels_ord] = pd.DataFrame(ord_enc.inverse_transform(knn_imp_df[labels_ord]), columns=labels_ord)
knn_imp_df.head()

,share_with_f&f,num_employees,ever_diagnosed,identified,identifited_neg_consequences,medical_coverage,primarily_tech,primary_role_tech,self-employed,sought_treatment,age
0,5,100-500,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,27.0
1,4,100-500,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,31.0
2,5,6-25,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,36.0
3,10,More than 1000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,22.0
4,10,500-1000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,52.0


In [120]:
# Impute using the most common answer for the categorical variables:
cat_df = data[labels_cat]
common = SimpleImputer(strategy = "most_frequent")
comm_imp = pd.DataFrame(common.fit_transform(cat_df), columns = labels_cat)

In [121]:
comm_imp.head()

,family_history,mention_interview,know_resources,emp_mh_resources,mh_benefits,mh_with_coworkers,mh_with_supervisor,past_disorder,ph_or_mh_discuss,reveal_to_clients
0,No,No,I know some,I don't know,No,Yes,Yes,Yes,Same level of comfort for each,Not applicable to me
1,No,No,I know some,No,Yes,Yes,Maybe,Yes,Same level of comfort for each,Not applicable to me
2,Yes,No,I know some,No,I don't know,Maybe,Yes,Yes,Same level of comfort for each,Not applicable to me
3,I don't know,No,I know some,I don't know,Yes,Yes,Yes,Yes,Same level of comfort for each,Not applicable to me
4,Yes,No,"Yes, I know several",No,Yes,Maybe,Yes,Yes,Physical health,"Sometimes, if it comes up"


## Save the data in new CSV for new models:
- categorical data imputed with the most frequent category
- ordinal data and boolean data imputed with KNN nearest neighbors, n=3 (together)

In [122]:
full_imputed_data1 = pd.concat([data[labels_left_alone], comm_imp, knn_imp_df], axis=1)
print(full_imputed_data1.shape)
full_imputed_data1.to_csv("data/imputed_data_r1.csv", index=False)

(1525, 29)


## Trying to do KNN on categorical data:

In [123]:
cat_df = data[labels_cat]
one_hot_cat = pd.get_dummies(cat_df)

# Convert rows where no column == 1 to np.nan:
num_categories = [len(cat_df[label].dropna().unique()) for label in cat_df.columns]
all_categories = one_hot_cat.columns

j = 2 # place within num_categories 
i = num_categories[0]+num_categories[1] # starting place, skip the first two labels (no NaN)

for label in cat_df.columns[2:]:
    group = all_categories[i:i+num_categories[j]]
    print(group)
    one_hot_cat.loc[one_hot_cat.iloc[:,i:i+num_categories[j]].sum(axis=1)==0, group] = np.nan
    i = i+num_categories[j]
    j +=1 

Index(['know_resources_I know some', 'know_resources_No, I don't know any',
       'know_resources_Yes, I know several'],
      dtype='object')
Index(['emp_mh_resources_I don't know', 'emp_mh_resources_No',
       'emp_mh_resources_Yes'],
      dtype='object')
Index(['mh_benefits_I don't know', 'mh_benefits_No',
       'mh_benefits_Not eligible for coverage / NA', 'mh_benefits_Yes'],
      dtype='object')
Index(['mh_with_coworkers_Maybe', 'mh_with_coworkers_No',
       'mh_with_coworkers_Yes'],
      dtype='object')
Index(['mh_with_supervisor_Maybe', 'mh_with_supervisor_No',
       'mh_with_supervisor_Yes'],
      dtype='object')
Index(['past_disorder_Don't Know', 'past_disorder_No',
       'past_disorder_Possibly', 'past_disorder_Yes'],
      dtype='object')
Index(['ph_or_mh_discuss_Mental health', 'ph_or_mh_discuss_Physical health',
       'ph_or_mh_discuss_Same level of comfort for each'],
      dtype='object')
Index(['reveal_to_clients_No, because it doesn't matter',
       'reveal

In [124]:
# Now use KNN imputation and then round back to get the categories:
cat_imputed = KNNImputer(n_neighbors=3).fit_transform(one_hot_cat)
# Convert back to df
cat_imp_df_hot = pd.DataFrame(cat_imputed.round(), columns = one_hot_cat.columns)

In [29]:
cat_imp_df_hot

,family_history_I don't know,family_history_No,family_history_Yes,mention_interview_Maybe,mention_interview_No,mention_interview_Yes,know_resources_I know some,"know_resources_No, I don't know any","know_resources_Yes, I know several",emp_mh_resources_I don't know,...,past_disorder_Possibly,past_disorder_Yes,ph_or_mh_discuss_Mental health,ph_or_mh_discuss_Physical health,ph_or_mh_discuss_Same level of comfort for each,"reveal_to_clients_No, because it doesn't matter","reveal_to_clients_No, because it would impact me negatively",reveal_to_clients_Not applicable to me,"reveal_to_clients_Sometimes, if it comes up","reveal_to_clients_Yes, always"
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1521,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1522,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1523,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
# Show that either the rows sum to 1 or 0, so there are no decimals and some rows are just 0s 
j = 0 # place within num_categories 
i = 0 # starting place

for label in cat_df.columns:
    print(i)
    group = all_categories[i:i+num_categories[j]]
    print(group)
    one = cat_imp_df_hot.loc[cat_imp_df_hot.iloc[:,i:i+num_categories[j]].sum(axis=1)!=0, group]
    zero = cat_imp_df_hot.loc[cat_imp_df_hot.iloc[:,i:i+num_categories[j]].sum(axis=1)!=1, group]
    print(len(one), len(zero))
    print(len(one)+len(zero), "\n")
    i = i+num_categories[j]
    j +=1 

0
Index(['family_history_I don't know', 'family_history_No',
       'family_history_Yes'],
      dtype='object')
1525 0
1525 

3
Index(['mention_interview_Maybe', 'mention_interview_No',
       'mention_interview_Yes'],
      dtype='object')
1525 0
1525 

6
Index(['know_resources_I know some', 'know_resources_No, I don't know any',
       'know_resources_Yes, I know several'],
      dtype='object')
1491 34
1525 

9
Index(['emp_mh_resources_I don't know', 'emp_mh_resources_No',
       'emp_mh_resources_Yes'],
      dtype='object')
1501 24
1525 

12
Index(['mh_benefits_I don't know', 'mh_benefits_No',
       'mh_benefits_Not eligible for coverage / NA', 'mh_benefits_Yes'],
      dtype='object')
1525 0
1525 

16
Index(['mh_with_coworkers_Maybe', 'mh_with_coworkers_No',
       'mh_with_coworkers_Yes'],
      dtype='object')
1487 38
1525 

19
Index(['mh_with_supervisor_Maybe', 'mh_with_supervisor_No',
       'mh_with_supervisor_Yes'],
      dtype='object')
1504 21
1525 

22
Index(['past_dis

In [125]:
# Convert back to the categories:
i = 0
j = 0
cat_imp_df = pd.DataFrame()

for label in cat_df.columns:
    group = all_categories[i:i+num_categories[j]]
    
    # idxmax() picks the first instance of the maximum value, so when all 0s picks the first item 
    grp_df = cat_imp_df_hot[group].idxmax(axis=1).str.replace(label+"_","")  
    cat_imp_df = pd.concat([cat_imp_df, grp_df], axis=1)
    i = i+num_categories[j]
    j += 1 
    
cat_imp_df.columns = labels_cat

In [98]:
cat_imp_df.isnull().sum()

family_history        0
mention_interview     0
know_resources        0
emp_mh_resources      0
mh_benefits           0
mh_with_coworkers     0
mh_with_supervisor    0
past_disorder         0
ph_or_mh_discuss      0
reveal_to_clients     0
dtype: int64

## Save the second set of data in new CSV for new models:
- categorical data one-hot encoded then KNN imputed and converted back
- ordinal data and boolean data imputed with KNN nearest neighbors, n=3 (together)

In [126]:
full_imputed_data2 = pd.concat([data[labels_left_alone], cat_imp_df, knn_imp_df], axis=1)
full_imputed_data2.to_csv("data/imputed_data_r2.csv")

In [127]:
full_imputed_data2.shape

(1525, 29)

## Third dataset- KNN imputation with all features together

In [128]:
one_hot_cat_cols = one_hot_cat.columns.to_list()
knn_data = pd.concat([one_hot_cat, ord_df_enc, bool_df, real_df], axis=1)
knn_imputed = KNNImputer(n_neighbors=3).fit_transform(knn_data)
knn_imp_df = pd.DataFrame(knn_imputed.round(), columns = one_hot_cat_cols+labels_ord+labels_bool+labels_real)

In [129]:
knn_imp_df.head()

,family_history_I don't know,family_history_No,family_history_Yes,mention_interview_Maybe,mention_interview_No,mention_interview_Yes,know_resources_I know some,"know_resources_No, I don't know any","know_resources_Yes, I know several",emp_mh_resources_I don't know,...,num_employees,ever_diagnosed,identified,identifited_neg_consequences,medical_coverage,primarily_tech,primary_role_tech,self-employed,sought_treatment,age
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,27.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,3.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,31.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,36.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,5.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,22.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,52.0


In [130]:
# Convert back to the categories:
i = 0
j = 0
cat_imp_df_hot = knn_imp_df[one_hot_cat_cols]
cat_imp_df = pd.DataFrame()

for label in cat_df.columns:
    group = all_categories[i:i+num_categories[j]]
    
    # idxmax() picks the first instance of the maximum value, so when all 0s picks the first item 
    grp_df = cat_imp_df_hot[group].idxmax(axis=1).str.replace(label+"_","")  
    cat_imp_df = pd.concat([cat_imp_df, grp_df], axis=1)
    i = i+num_categories[j]
    j += 1 
    
cat_imp_df.columns = labels_cat

In [ ]:
knn_imp_df.head

In [131]:
# combine the categorical imputed with the rest of the imputed:
full_imputed_data3 = pd.concat([data[labels_left_alone], cat_imp_df, knn_imp_df])
full_imputed_data3.to_csv("data/imputed_data_r3.csv")
full_imputed_data2.shape

ValueError: all arrays must be same length